In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from adan_pytorch import Adan
import copy
from utils import preparte_data_loader
from utils import set_parameter_requires_grad
from utils import DEVICE, LOSS_CRITERIA
import time
from transformers import HubertConfig, HubertForSequenceClassification
from utils import ONLY_10_LABELS
import matplotlib.pyplot as plt
from utils import DEVICE, AudioDataset
from torch.utils.data import DataLoader
import optuna
import pandas as pd

/home/mlspeech/royweber/anaconda3/envs/roy_lang/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
batch_size = 64
num_labels = 50#10 if ONLY_10_LABELS else 50
epochs = 10
device=DEVICE
loss_criteria = LOSS_CRITERIA
# train_dataloader = preparte_data_loader(mode='train', batch_size=20)
# val_dataloader = preparte_data_loader(mode='val', batch_size=20)
train_dataset = AudioDataset(kind='train')
val_dataset = AudioDataset(kind='val')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

dataloaders = {
    'train':train_loader,
    'val':val_loader
}



def optune_optimizer_for_model(trial, optimizer_name):
    
    config = HubertConfig(
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=4,
    intermediate_size=256,
    num_labels=num_labels,
    conv_dim=(512, 512, 512),
    conv_stride=(5, 2, 2),
    conv_kernel=(10, 3, 3)
    )

    hubert_model = HubertForSequenceClassification(config)
    
    model = hubert_model.to(DEVICE)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)  # log=True, will use log scale to interplolate between lr
    #trial.suggest_categorical("optimizer", ["Adam", "SGD","Adan"])
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == 'SGD':
        sgd_momentum = trial.suggest_float("sgd_momentum", 1e-1, 1)
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=sgd_momentum, nesterov=True)
    elif optimizer_name == 'Adan':
        beta1 = trial.suggest_float("beta1", 1e-3, 1e-1)
        beta2 = trial.suggest_float("beta2", 1e-3, 1e-1)
        beta3 = trial.suggest_float("beta3", 1e-3, 1e-1)
        optimizer = Adan(model.parameters(),lr = lr,
            betas = (beta1, beta2, beta3), 
            weight_decay = 0.02         # weight decay 0.02 is optimal per author
        )
        
    
    # out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)  # number of units will be between 4 and 128
    

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)
        
        model.train()
        
        # Iterate over data.
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(True):
                # Get model outputs and calculate loss
                logits_h = model(inputs).logits.type(torch.cuda.FloatTensor)
                labels = labels.type(torch.cuda.FloatTensor)
                loss = loss_criteria(logits_h.view(-1), labels.view(-1))
                # loss = loss_criteria(outputs, labels)

                _, preds = torch.max(logits_h, 1)
                _, real_labels = torch.max(labels, 1)

                # backward + optimize only if in training phase
                # zero the parameter gradients
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
        model.eval()
        running_corrects = 0.0
        for inputs, labels in dataloaders['val']:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(False):
                    # Get model outputs and calculate loss
                    # outputs = model(inputs)
                    
                    logits_h = model(inputs).logits.type(torch.cuda.FloatTensor)
                    labels = labels.type(torch.cuda.FloatTensor)
                    loss = loss_criteria(logits_h.view(-1), labels.view(-1))
                    # loss = loss_criteria(outputs, labels)

                    _, preds = torch.max(logits_h, 1)
                    _, real_labels = torch.max(labels, 1)
                    
                    # loss = loss_criteria(outputs, labels)
                    # _, preds = torch.max(outputs, 1)

                # statistics
                running_corrects += torch.sum(preds == real_labels.data)

        epoch_acc = running_corrects.double() / len(dataloaders['val'].dataset)

        # report back to Optuna how far it is (epoch-wise) into the trial and how well it is doing (accuracy)
        trial.report(epoch_acc, epoch)  

        # then, Optuna can decide if the trial should be pruned
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return epoch_acc

In [9]:
# now we can run the experiment
sampler = optuna.samplers.TPESampler()
configurations = [{'optimizer_name':'Adam',
                   'study_name':'adam_study'},
                  {'optimizer_name':'SGD',
                   'study_name':'sgd_study'},
                  {'optimizer_name':'Adan',
                   'study_name':'adan_study'}
                  ]
studies = {}
for config in configurations:
    
    study = optuna.create_study(study_name=config['study_name'], direction="maximize", sampler=sampler)
    study.optimize(lambda trial: optune_optimizer_for_model(trial, config['optimizer_name']), n_trials=500, timeout=600)
    
    study_csv = f'{config["study_name"]}.csv'
    study.trials_dataframe().to_csv(study_csv)
    studies[config["study_name"]] = study_csv
    
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))
    print("Best trial:")

    trial = study.best_trial

    print("  Value: ", trial.value)
    print("  Params: ")

    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-08-07 22:01:05,634] A new study created in memory with name: adam_study


Epoch 0/9
----------
Epoch 1/9
----------
Epoch 2/9
----------
Epoch 3/9
----------
Epoch 4/9
----------
Epoch 5/9
----------
Epoch 6/9
----------
Epoch 7/9
----------
Epoch 8/9
----------
Epoch 9/9
----------


[I 2024-08-07 22:02:11,162] Trial 0 finished with value: 0.1 and parameters: {'lr': 0.023412811673960664}. Best is trial 0 with value: 0.1.


Epoch 0/9
----------
Epoch 1/9
----------
Epoch 2/9
----------
Epoch 3/9
----------
Epoch 4/9
----------
Epoch 5/9
----------
Epoch 6/9
----------
Epoch 7/9
----------
Epoch 8/9
----------
Epoch 9/9
----------


[I 2024-08-07 22:03:07,826] Trial 1 finished with value: 0.1 and parameters: {'lr': 0.03933215121713092}. Best is trial 0 with value: 0.1.


...

Epoch 0/9
----------


[I 2024-08-07 22:31:25,157] Trial 16 pruned. 


Epoch 0/9
----------
Epoch 1/9
----------
Epoch 2/9
----------
Epoch 3/9
----------
Epoch 4/9
----------


[I 2024-08-07 22:31:58,267] Trial 17 pruned. 


Study statistics: 
  Number of finished trials:  18
  Number of pruned trials:  11
  Number of complete trials:  7
Best trial:
  Value:  0.5208333333333334
  Params: 
    lr: 0.0020022613730624438
    beta1: 0.046914599109421654
    beta2: 0.019381017240822685
    beta3: 0.010655285691959324


In [ ]:
#ADAM: 
# Params: 
#     lr: 0.0012161288828451268
# SGD
# lr: 0.0004399074848905889
#     sgd_momentum: 0.782315314217227

# Params Adan: 
#     lr: 0.0020022613730624438
#     beta1: 0.046914599109421654
#     beta2: 0.019381017240822685
#     beta3: 0.010655285691959324

In [10]:
studies

{'adam_study': 'adam_study.csv',
 'sgd_study': 'sgd_study.csv',
 'adan_study': 'adan_study.csv'}

In [12]:
study.trials_dataframe().to_csv('study_trials.csv')

In [22]:

def recover_study(file_name='study_trials.csv'):
    trials_df = pd.read_csv(file_name)
    study = optuna.create_study()

    # Add trials back to the study
    for _, row in trials_df.iterrows():
        trial_params = {col.replace('params_',''): row[col] for col in trials_df.columns if col.startswith('params_')}
        trial = optuna.trial.create_trial(
            params=trial_params,
            distributions={
            'lr': optuna.distributions.FloatDistribution(1e-5, 1e-1),
            'beta1': optuna.distributions.FloatDistribution(1e-3, 1e-1),
            'beta2': optuna.distributions.FloatDistribution(1e-3, 1e-1),
            'beta3': optuna.distributions.FloatDistribution(1e-3, 1e-1)
            },
            value=row['value'],
            state=optuna.trial.TrialState.COMPLETE
        )
        # lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
        study.add_trial(trial)
    return study



In [23]:
for study_name, study_file in studies.items():
    if  study_name == 'adan_study':
        study = recover_study(study_file)
        break
optuna.visualization.plot_param_importances(study)

[I 2024-08-07 22:48:41,163] A new study created in memory with name: no-name-c820f13d-0507-4c91-a775-02d70e60a53c


In [24]:
optuna.visualization.plot_contour(study, params=["beta3","lr"])

In [ ]:
#process - 
#1. show usage
#2. 
#3 taking model from scratch - 
# finetuning - 
#4 small bert - 
#5 load from scratch -

#just if we train from zero we can see the difference
#

#main question - optuna on large data 